<a href="https://colab.research.google.com/github/Kerriea-star/TensorFlow-Customization/blob/main/Custom_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We recommend using `tf.keras` as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution

In [1]:
import tensorflow as tf

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full Keras API in the tf.keras package, and the Keras layers are very useful when building your own models.

In [3]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [4]:
# To use a layer simply call it
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
# layers have many useful methods. For example, you can inspect all variables
# in a layer using 'layer.variables' and trainable variables using
# 'layer.trainable_variables'. In this case a fully-connected layer
# will have variables for weights and biases
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.41538447, -0.39552945, -0.43737733,  0.21807867,  0.01221454,
         -0.21406072,  0.05848324, -0.42509842, -0.3170494 , -0.49346277],
        [ 0.02651763, -0.18987685,  0.0649904 , -0.48276067, -0.36741722,
         -0.46315822,  0.4911068 ,  0.47339648, -0.33553058, -0.1682427 ],
        [-0.50481737, -0.1833094 ,  0.48243374,  0.46239668,  0.2577306 ,
         -0.24608728,  0.32431293,  0.3103972 , -0.07480168,  0.53747123],
        [ 0.18960094, -0.08060372,  0.62434536, -0.5326756 ,  0.04835325,
         -0.56158084, -0.13228524, -0.43877125,  0.5967962 , -0.11114728],
        [ 0.5108629 , -0.28678828,  0.5370893 ,  0.2438348 ,  0.48180765,
          0.5346511 ,  0.43981093,  0.29933393,  0.3932442 , -0.05056775]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [6]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.41538447, -0.39552945, -0.43737733,  0.21807867,  0.01221454,
         -0.21406072,  0.05848324, -0.42509842, -0.3170494 , -0.49346277],
        [ 0.02651763, -0.18987685,  0.0649904 , -0.48276067, -0.36741722,
         -0.46315822,  0.4911068 ,  0.47339648, -0.33553058, -0.1682427 ],
        [-0.50481737, -0.1833094 ,  0.48243374,  0.46239668,  0.2577306 ,
         -0.24608728,  0.32431293,  0.3103972 , -0.07480168,  0.53747123],
        [ 0.18960094, -0.08060372,  0.62434536, -0.5326756 ,  0.04835325,
         -0.56158084, -0.13228524, -0.43877125,  0.5967962 , -0.11114728],
        [ 0.5108629 , -0.28678828,  0.5370893 ,  0.2438348 ,  0.48180765,
          0.5346511 ,  0.43981093,  0.29933393,  0.3932442 , -0.05056775]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## Implementing custom layers

The best way to implement your own layer is extending the tf.keras.Layer class and implementing:

1.  ` __init__` , where you can do all input-independent initialization
2.   `build`, where you know the shapes of the input tensors and can do the rest of the initialization
3.   `call`, where you do the forward computation

Note that you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`. However, the advantage of creating them in `build` is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in `__init__` would mean that shapes required to create the variables will need to be explicitly specified.


In [9]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [10]:
_ = layer(tf.zeros([10, 5])) # calling the layer '.builds' it.

In [11]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer_1/kernel:0']


## Models: Composing layers

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

Typically you inherit from `keras.Model` when you need the model methods like: `Model.fit`, `Model.evaluate`,  and `Model.save`.

One other feature provided by `keras.Model` (instead of `keras.layers.Layer`) is that in addition to tracking variables, a `keras.Model` also tracks its internal layers, making them easier to inspect.

For example here is a ResNet block:

In [14]:
class RestNetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(RestNetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)

block = RestNetIdentityBlock(1, [1, 2, 3])

In [15]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [16]:
block.layers

In [17]:
len(block.variables)

18

In [18]:
block.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization_3 (Bat  multiple                  4         
 chNormalization)                                                
                                                                 
 conv2d_4 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization_4 (Bat  multiple                  8         
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           multiple                  9         
                                                                 
 batch_normalization_5 (Bat  multiple                  12        
